In [1]:
!pip install -U langchain-community pandas numpy scikit-learn faiss-gpu sentence-transformers huggingface_hub


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 84.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 20.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.9/275.9 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.0/468.0 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 414.3/414.3 kB 21.6 MB/s eta 0:00:00
  Attempting uninstall: async-timeout
    Found existing installation: async-timeout 5.0.1
    Uninstalling async-timeout-5.0.1:
      Successfully uninstalled async-timeout-5.0.1
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.29.0
    Uninstalling huggingface-hub-0.29.0:
      Successful

In [ ]:
import pandas as pd
import numpy as np
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import DataFrameLoader
from langchain.llms import HuggingFaceHub
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.vectorstores import FAISS
from langchain.llms import HuggingFaceEndpoint

import os
import gc
import re


In [ ]:
# Set the Hugging Face API token
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "Replace with your own API KEY" 

In [ ]:
DATA_PATH = "/kaggle/input/bookcrossing-dataset/Books Data with Category Language and Summary/Preprocessed_data.csv"

try:
    prepro_data = pd.read_csv(DATA_PATH)
except FileNotFoundError:
    print("Error: Dataset not found at", DATA_PATH)
    exit()



In [ ]:
"""Filtering and preparing the dataset"""

new_book = prepro_data.drop(columns=['Unnamed: 0', 'location', 'age', 'year_of_publication', 'publisher', 'img_s', 'img_l',
                                     'img_m', 'Language', 'city', 'state', 'country','user_id','isbn','rating'])

new_book['book_author'] = new_book['book_author'].fillna('Unknown')

# let's apply regex to make thing simple
#let's clean the book title
def clean_book_title(title):
    return re.sub("[^a-zA-Z0-9 ]", "",title)
new_book['clean_book_title'] = new_book['book_title'].apply(clean_book_title)

new_unique_book = new_book.drop_duplicates(subset=['clean_book_title','book_author','Summary'])

new_unique_book = new_unique_book[['clean_book_title', 'Category','Summary', 'book_author']]

new_unique_book.columns = ['book_title', 'Category','Summary', 'book_author'] 


In [ ]:
new_unique_book.head(2)

,book_title,Category,Summary,book_author
0,Classical Mythology,['Social Science'],Provides an introduction to classical myths pl...,Mark P. O. Morford
1,Clara Callan,['Actresses'],"In a small town in Canada, Clara Callan reluct...",Richard Bruce Wright


In [ ]:
# Combines the title and the summary together.
def combine_title_summary(row):
    return f"{row['book_title']} {row['Summary']}" 

new_unique_book['combined_text'] = new_unique_book.apply(combine_title_summary, axis=1)


SAMPLE_SIZE = 1000  
df_sampled = new_unique_book.sample(n=SAMPLE_SIZE, random_state=42)

In [8]:
df_sampled.head(2)

,book_title,Category,Summary,book_author,combined_text
849808,Young Indiana Jones and the Princess of Peril,['Adventure and adventurers'],"In 1913 Russia, Indy befriends a young Georgia...",Les Martin,Young Indiana Jones and the Princess of Peril ...
884496,Once upon a Time,['Fiction'],Prince Ruyen of the Isle of Falcon Bruine has ...,Constance O'Banyon,Once upon a Time Prince Ruyen of the Isle of F...


In [ ]:
"""This Embedding is not giving us good result"""

# Create Book Embeddings
print("Creating Book Embeddings...")
model_name = "all-mpnet-base-v2" 

encode_kwargs = {'normalize_embeddings': False}
hf_embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    encode_kwargs=encode_kwargs
)


Creating Book Embeddings...


<ipython-input-9-95501513d37f>:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf_embeddings = HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:


#Documents for FAISS
loader = DataFrameLoader(df_sampled, page_content_column="combined_text")
documents = loader.load()

#Store Embeddings in FAISS
print("Storing Embeddings in FAISS...")
db = FAISS.from_documents(documents, hf_embeddings)

#clean up memory
del prepro_data, new_book, new_unique_book, df_sampled, loader, documents
gc.collect()

Storing Embeddings in FAISS...


78

In [ ]:
import time
def recommend_books(user_input, num_recommendations=3):
    """
    Recommends books based on user input using embeddings for similarity search
    and an LLM (via Hugging Face Inference API) to generate recommendations.
    """
    print("Generating Recommendations...")

    # Find similar books using FAISS embeddings
    similar_books = db.similarity_search(user_input, k=5)
    book_titles = "\n".join([f"Title: {doc.page_content}" for doc in similar_books])
    
    print("--- FAISS Similar Books ---")
    print(book_titles)
    print("--- End FAISS Similar Books ---")
    
    print("Accessing HF LLM")   
    
    
    repo_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0" # "google/flan-t5-small" 
    llm = HuggingFaceEndpoint(
    repo_id=repo_id,
    temperature=0.6,
    max_new_tokens=150, 
    wait_for_model=True)



    
    prompt_template = """
    You are a book recommendation expert. The user likes: {user_input}

    Here are some books similar to what the user likes:
    {book_titles}

    Based on this information, recommend THREE books. For EACH book, provide the title and a VERY BRIEF (one-sentence) explanation of why the user might like it.
    Be concise and engaging. Do NOT include website URLs or marketing content. Do NOT recommend books already listed.
    For each book, provide the title and the summary.

    Example Recommendations:
    - "The Name of the Wind (Summary: A beautifully written fantasy novel with a compelling protagonist.): A fantastical adventure"
    - "Pride and Prejudice (Summary: A classic romance with witty dialogue and memorable characters.): A romance story"
    - "The Martian (Summary: A thrilling science fiction story about survival against all odds.): Sci-fi."

    Recommendations:
    """
    
    prompt = PromptTemplate(template=prompt_template,
                              input_variables=["user_input", "book_titles"])
    
    llm_chain = LLMChain(prompt=prompt, llm=llm)
    
    # Retry mechanism in case of transient 503 errors
    max_retries = 3
    for attempt in range(max_retries):
        try:
            recommendations = llm_chain.run({"user_input": user_input, "book_titles": book_titles})
            break 
        except Exception as e:
            print(f"Attempt {attempt+1} failed with error: {e}")
            time.sleep(2)  
            if attempt == max_retries - 1:
                return "Sorry, I encountered an error generating recommendations. Please try again later."
    
    return recommendations


In [ ]:
#Get User Input and Print Recommendations
user_input = input("Enter a book you like, a genre, or a topic you're interested in: ")
print("Running. Please wait...")

recommendations = recommend_books(user_input)
print("\nRecommendations:")
print(recommendations)


Enter a book you like, a genre, or a topic you're interested in:  time travel


Running. Please wait...
Generating Recommendations...
--- FAISS Similar Books ---
Title: You See the Future Choose Your Own Adventure No 44 The reader&#39;s decisions will determine what happens when a bump on
the head leaves the reader with the ability to see the future.
Title: A question of time A Red badge novel of suspense 9
Title: The Theory of Relativity  Other Essays 9
Title: Serendipity 9
Title: Tarot 9
--- End FAISS Similar Books ---
Accessing HF LLM


<ipython-input-14-7289cd463f97>:24: LangChainDeprecationWarning: The class `HuggingFaceEndpoint` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  llm = HuggingFaceEndpoint(
<ipython-input-14-7289cd463f97>:54: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(prompt=prompt, llm=llm)
<ipython-input-14-7289cd463f97>:60: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  recommendations = llm_chain.run({"user_input": user_input, "book_titles": book_titles})
/usr/local/lib/python3.10/dis


Recommendations:
1. "The Name of the Wind" by Patrick Rothfuss
    2. "Pride and Prejudice" by Jane Austen
    3. "The Martian" by Andy Weir

I hope that helps! Let me know if you have any other questions.
